# Programming Problem 1: HMM for NER (30 points)
Welcome to the programming portion of the assignment! Each assignment throughout the semester will have a written portion and a programming portion. We will be using [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true), so if you have never used it before, take a quick look through this introduction: [Working with Google Colab](https://docs.google.com/document/d/1LlnXoOblXwW3YX-0yG_5seTXJsb3kRdMMRYqs8Qqum4/edit?usp=sharing).

### Writing Code
Look for the keyword "TODO" and fill in your code in the empty space.
Feel free to add and delete arguments in function signatures, but be careful that you might need to change them in function calls which are already present in the notebook.

### Data preprocessing

In this section we will write code to load data and build the dataset for Named Entity Recognition.

In [1]:
import os
import re
import codecs

Write a function to load sentences.

In [2]:
def zero_digits(s):
    """
    Replace all digits in a string with zeros.
    """
    return re.sub('\d', '0', s)

def load_sentences(path):
    """
    Load sentences. A line must contain at least a word and its tag.
    Sentences are separated by empty lines.
    """
    sentences = []
    sentence = []
    for line in codecs.open(path, 'r', 'utf8'):
        line = zero_digits(line.rstrip())
        if not line:
            if len(sentence) > 0:
                if 'DOCSTART' not in sentence[0][0]:
                    sentences.append(sentence)
                sentence = []
        else:
            word = line.split()
            assert len(word) >= 2
            sentence.append(word)
    if len(sentence) > 0:
        if 'DOCSTART' not in sentence[0][0]:
            sentences.append(sentence)
    return sentences

In [3]:
# Example on the usage of zero_digits::
text = "My phone number is 123-456-7890."
result = zero_digits(text)
print(result)


My phone number is 000-000-0000.


Prepare the training/test data using the loaded sentences.

In [5]:
def create_dico(item_list):
    """
    Create a dictionary of items from a list of list of items.
    """
    assert type(item_list) is list
    dico = {}
    for items in item_list:
        for item in items:
            if item not in dico:
                dico[item] = 1
            else:
                dico[item] += 1
    return dico

In [6]:
# Test the create_dico function
item_list = [
    ["apple", "banana", "apple"],
    ["banana", "orange"],
    ["apple", "orange", "banana", "banana"]
]

dicto = create_dico(item_list)
print(dicto)


{'apple': 3, 'banana': 4, 'orange': 2}


In [7]:
def create_mapping(dico):
    """
    Create a mapping (item to ID / ID to item) from a dictionary.
    Items are ordered by decreasing frequency.
    """
    sorted_items = sorted(dico.items(), key=lambda x: (-x[1], x[0]))
    id_to_item = {i: v[0] for i, v in enumerate(sorted_items) if v[1] > 2}
    item_to_id = {v: k for k, v in id_to_item.items()}
    return item_to_id, id_to_item

def word_mapping(sentences):
    """
    Create a dictionary and a mapping of words, sorted by frequency.
    """
    words = [[x[0] for x in s] for s in sentences]
    dico = create_dico(words)
    dico['<UNK>'] = 10000000
    word_to_id, id_to_word = create_mapping(dico)
    print("Found %i unique words (%i in total)" % (
        len(dico), sum(len(x) for x in words))
    )
    return dico, word_to_id, id_to_word

def tag_mapping(sentences):
    """
    Create a dictionary and a mapping of tags, sorted by frequency.
    """
    tags = [[word[-1] for word in s] for s in sentences]
    dico = create_dico(tags)
    tag_to_id, id_to_tag = create_mapping(dico)
    print("Found %i unique named entity tags" % len(dico))
    return dico, tag_to_id, id_to_tag

In [8]:
# Sample sentence dataset (word-tag format)
sentences = [
    [["John", "B-PER"], ["loves", "O"], ["Mary", "B-PER"]],
    [["Paris", "B-LOC"], ["is", "O"], ["beautiful", "O"]],
    [["London", "B-LOC"], ["is", "O"], ["great", "O"]],
    [["John", "B-PER"], ["visits", "O"], ["London", "B-LOC"]],
]

# Test word_mapping function
dico_words, word_to_id, id_to_word = word_mapping(sentences)
print("Word Dictionary:", dico_words)
print("Word to ID Mapping:", word_to_id)
print("ID to Word Mapping:", id_to_word)

# Test tag_mapping function
dico_tags, tag_to_id, id_to_tag = tag_mapping(sentences)
print("Tag Dictionary:", dico_tags)
print("Tag to ID Mapping:", tag_to_id)
print("ID to Tag Mapping:", id_to_tag)


Found 10 unique words (12 in total)
Word Dictionary: {'John': 2, 'loves': 1, 'Mary': 1, 'Paris': 1, 'is': 2, 'beautiful': 1, 'London': 2, 'great': 1, 'visits': 1, '<UNK>': 10000000}
Word to ID Mapping: {'<UNK>': 0}
ID to Word Mapping: {0: '<UNK>'}
Found 3 unique named entity tags
Tag Dictionary: {'B-PER': 3, 'O': 6, 'B-LOC': 3}
Tag to ID Mapping: {'O': 0, 'B-LOC': 1, 'B-PER': 2}
ID to Tag Mapping: {0: 'O', 1: 'B-LOC', 2: 'B-PER'}


In [9]:
def prepare_dataset(sentences, mode=None, word_to_id=None, tag_to_id=None):
    """
    Prepare the dataset. Return 'data', which is a list of lists of dictionaries containing:
        - words (strings)
        - word indexes
        - tag indexes
    """
    assert mode == 'train' or (mode == 'test' and word_to_id and tag_to_id)

    if mode=='train':
        word_dic, word_to_id, id_to_word = word_mapping(sentences)
        tag_dic, tag_to_id, id_to_tag = tag_mapping(sentences)

    def f(x): return x
    data = []
    for s in sentences:
        str_words = [w[0] for w in s]
        words = [word_to_id[f(w) if f(w) in word_to_id else '<UNK>']
                 for w in str_words]
        tags = [tag_to_id[w[-1]] for w in s]
        data.append({
            'str_words': str_words,
            'words': words,
            'tags': tags,
        })

    if mode == 'train':
        return data, {'word_to_id':word_to_id, 'id_to_word':id_to_word, 'tag_to_id':tag_to_id, 'id_to_tag':id_to_tag}
    else:
        return data

### Hidden Markov Model
In this section, we will implement a bigram hidden markov model (HMM) that could perform two types of decoding: greedy decoding and viterbi decoding.


## Hidden Markov Model for Named Entity Recognition

In this section, we implement a bigram Hidden Markov Model (HMM) for Named Entity Recognition. HMMs are probabilistic sequence models that capture the underlying structure of sequential data.

### Components of HMM

An HMM is defined by the following components:

1. **States**: In our context, these are the named entity tags (e.g., O, PER, LOC, etc.)
2. **Observations**: These are the words in our sentences
3. **Three probability distributions**:

#### 1. Initial probabilities: $\pi_i = P(t_1 = i)$
- The probability that the first tag in a sentence is tag $i$
- Computed by counting how many times tag $i$ appears as the first tag in sentences in the training corpus, divided by the total number of sentences.
- Formula: $\pi_i = \frac{\text{Count of sentences starting with tag } i}{\text{Total number of sentences}}$

#### 2. Transition probabilities: $A_{i,j} = P(t_n = j | t_{n-1} = i)$
- The probability of transitioning from tag $i$ to tag $j$
- Computed by counting how many times tag $j$ follows tag $i$ in the training corpus, divided by the total number of times tag $i$ appears.
- Formula: $A_{i,j} = \frac{\text{Count of tag } j \text{ following tag } i}{\text{Total occurrences of tag } i}$

#### 3. Emission probabilities: $B_{i,k} = P(w_n = k | t_n = i)$
- The probability of observing word $k$ given the tag $i$
- Computed by counting how many times word $k$ is associated with tag $i$ in the training corpus, divided by the total number of times tag $i$ appears.
- Formula: $B_{i,k} = \frac{\text{Count of word } k \text{ with tag } i}{\text{Total occurrences of tag } i}$

### Decoding Algorithms

We implement two decoding algorithms:

#### 1. Greedy Decoding
- For each position, choose the tag that maximizes the product of transition and emission probabilities
- At position $n$, select tag $j$ that maximizes $A_{i,j} \times B_{j,w_n}$ where $i$ is the tag at position $n-1$
- Simple but doesn't guarantee the optimal sequence

#### 2. Viterbi Decoding
- Dynamic programming algorithm that finds the globally optimal tag sequence
- Uses a matrix $M$ where $M[n,j]$ represents the probability of the most likely path ending with tag $j$ at position $n$
- Recursive formula: $M[n,j] = \max_i(M[n-1,i] \times A_{i,j} \times B_{j,w_n})$
- Additionally, we use a backpointer matrix $B$ to reconstruct the optimal path
- The final sequence is obtained by tracing back through the backpointer matrix from the most probable final state

### Implementation Details

- We use Maximum Likelihood Estimation (MLE) for training the model
- To avoid numerical underflow issues in Viterbi, we work in the log space for probability calculations
- The emission and transition probabilities are normalized to ensure they form valid probability distributions

In [10]:
import os
import re
import numpy as np
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
import collections
import codecs

In [11]:
class HMM(object):
    """
     HMM Model
    """
    def __init__(self, dic, decode_type):
        """
        Initialize the model.
        """

        self.num_words = len(dic['word_to_id'])
        self.num_tags = len(dic['tag_to_id'])

        self.initial_prob = np.ones([self.num_tags])
        self.transition_prob = np.ones([self.num_tags, self.num_tags])
        self.emission_prob = np.ones([self.num_tags, self.num_words])
        self.decode_type = decode_type

        return

    def train(self, corpus):
        """
        TODO: Train a bigram HMM model using MLE estimates.
        Complete the code to compute self,initial_prob, self.transition_prob and self.emission_prob appropriately.

        Args:
            corpus is a list of dictionaries of the form:
            {'str_words': str_words,   ### List of string words
            'words': words,            ### List of word IDs
            'tags': tags}              ### List of tag IDs
            All three lists above have length equal to the sentence length for each instance.

        """

        # Step 1: Compute initial_probs.
        # initial_prob[x]: the probability of tag x to be assigned to the first token in a sentence.
        self.initial_prob = np.zeros([self.num_tags])
        for sentence in corpus:
            # TODO: update self.initial_prob
            # (5 points)
            # START HERE
            first_tag=sentence['tags'][0]
            self.initial_prob[first_tag]+=1
            # END HERE

        # Normarlize initial_prob to sum to 1
        self.initial_prob /= np.sum(self.initial_prob)

        # Step 2: Complete the code to compute transition_prob.
        # transition_prob[x][y]: the probability of tag y to appear after tag x.
        self.transition_prob = np.zeros([self.num_tags, self.num_tags])
        for sentence in corpus:
            tag = sentence['tags']
            for i in range(1, len(tag)):
                # TODO: update self.transition_prob
                # (5 points)
                # START HERE
                prev_tag = tag[i-1]
                curr_tag = tag[i]
                self.transition_prob[prev_tag][curr_tag] += 1
                # END HERE

        # Normalize every row of transition_prob to sum to 1.
        for p in self.transition_prob:
            p /= np.sum(p)


        # Step 3: Complete the code to compute emission_prob
        # emission_prob[x][y]: the probability of word y to appear given tag x.
        # Note that for each sentence s in the corpus, word IDs are in s['words'].
        self.emission_prob = np.zeros([self.num_tags, self.num_words])
        for sentence in corpus:
            for i in range(len(sentence['tags'])):
                # TODO: update self.emission_prob
                # (5 points)
                # START HERE
                tag_id = sentence['tags'][i]
                word_id = sentence['words'][i]
                self.emission_prob[tag_id][word_id] += 1
                # END HERE

        # For every tag, normalize the emission_prob to sum to 1.
        for p in self.emission_prob:
            p /= np.sum(p)

        return

    def greedy_decode(self, sentence):
        """
        Decode a single sentence in Greedy fashion
        Return a list of tags.
        """
        tags = []

        init_scores = [self.initial_prob[t] * self.emission_prob[t][sentence[0]] for t in range(self.num_tags)]
        tags.append(np.argmax(init_scores))

        for w in sentence[1:]:
            scores = [self.transition_prob[tags[-1]][t] * self.emission_prob[t][w] for t in range(self.num_tags)]
            tags.append(np.argmax(scores))

        assert len(tags) == len(sentence)
        return tags

    def viterbi_decode(self, sentence):
        """
        TODO: Complete the code to decode a single sentence using the Viterbi algorithm.
        Args:
             sentence -- a list of ints that represents word IDs.
        Output:
             tags     -- a list of ints that represents the tags decoded from the input.
        """
        tags = []
        l = len(sentence)

        pi = self.initial_prob
        A = self.transition_prob
        O = self.emission_prob

        # Let M be the state matrix.
        # M[i,j]: most probable sequence of tags ending with tag j at the i-th token.
        M = np.zeros((l, self.num_tags))
        M[:,:] = float('-inf')

        # Use B to track the path to reach the most probable sequence.
        # B[i,j] is the tag of the (i-1)-th token in the most probable sequence ending with tag j at the i-th token.
        B = np.zeros((l, self.num_tags), 'int')

        # Compute the probability to assign each tag to the first token.
        M[0, :] = pi * O[:, sentence[0]]

        # Dynamic programming.
        for i in range(1, l):
            for j in range(self.num_tags):
                # TODO: Compute M[i, j] and B[i, j].
                # (10 points)
                # START HERE
                scores = M[i-1, :] + np.log(A[:, j]) + np.log(O[j, sentence[i]])
                best_tag = np.argmax(scores)
                M[i, j] = scores[best_tag]
                B[i, j] = best_tag
                # END HERE

        # Extract the optimal sequence of tags from B.
        # Start from the last position, and iteratively find the tag of each position that results in the most probable tag sequence.
        tags.append(np.argmax(M[l-1,:]))
        for i in range(l-1, 0, -1):
            # TODO: Extract the tag of the (i-1)-th token that results in the most probable sequence of tags.
            # (5 points)
            # START HERE
            prev_tag = B[i, tags[-1]]
            tags.insert(0, prev_tag)
            # END HERE

        return tags

    def tag(self, sentence):
        """
        Tag a sentence using a trained HMM.
        """
        if self.decode_type == 'viterbi':
            return self.viterbi_decode(sentence)
        elif self.decode_type == 'greedy':
            return self.greedy_decode(sentence)
        else:
            raise ValueError("Unknown decoding type")

### Train and evaluate HMMs.
This section contains driver code for learning a HMM for named entity recognition on a training corpus and evaluating it on a test corpus.

In [12]:
import os
import time
import codecs
import json
from tqdm import tqdm
import numpy as np
import collections
from sklearn.metrics import f1_score, confusion_matrix

Write a function to tag the test corpus with a trained model.

In [13]:
def tag_corpus(model, test_corpus, output_file, dic):
    if output_file:
        f_output = codecs.open(output_file, 'w', 'utf-8')
    start = time.time()

    num_correct = 0.
    num_total = 0.
    y_pred=[]
    y_actual=[]
    print('Tagging...')
    for i, sentence in enumerate(tqdm(test_corpus)):
        tags = model.tag(sentence['words'])
        str_tags = [dic['id_to_tag'][t] for t in tags]
        y_pred.extend(tags)
        y_actual.extend(sentence['tags'])

        # Check accuracy.
        num_correct += np.sum(np.array(tags) == np.array(sentence['tags']))
        num_total += len([w for w in sentence['words']])

        if output_file:
            f_output.write('%s\n' % ' '.join('%s%s%s' % (w, '__', y)
                                             for w, y in zip(sentence['str_words'], str_tags)))

    print('---- %i lines tagged in %.4fs ----' % (len(test_corpus), time.time() - start))
    if output_file:
        f_output.close()

    print("Overall accuracy: %s\n" % (num_correct/num_total))
    return y_pred,y_actual

Write a function a compute the confusion matrix and the F-1 score.

In [14]:
def compute_score(y_pred,y_actual):
    A = confusion_matrix(y_actual, y_pred)
    f1 = f1_score(y_actual, y_pred,average=None)
    print("Confusion Matrix:\n", A)
    print("F-1 scores: ", f1)

Write a function to train and evalute the model.

In [15]:
def runHiddenMarkovModel(train_corpus,
                         test_corpus,
                         dic,
                         decode_type,
                         output_file):
    # build and train the model
    model = HMM(dic, decode_type)
    model.train(train_corpus)

    print("Train results:")
    pred, real = tag_corpus(model, train_corpus, output_file, dic)

    print("Tags: ", dic['id_to_tag'])
    A = compute_score(pred,real)

    # test on validation
    print("\n-----------\nTest results:")
    pred, real = tag_corpus(model, test_corpus, output_file, dic)

    print("Tags: ", dic['id_to_tag'])
    A = compute_score(pred,real)

### Experiments

#### Load data

In [16]:
# Download the dataset
!wget https://princeton-nlp.github.io/cos484/assignments/a2/eng.train
!wget https://princeton-nlp.github.io/cos484/assignments/a2/eng.val

train_file = 'eng.train'
test_file = 'eng.val'

# Load the training data
train_sentences = load_sentences(train_file)
train_corpus, dic = prepare_dataset(train_sentences, mode='train', word_to_id=None, tag_to_id=None)

# Load the testing data
test_sentences = load_sentences(test_file)
test_corpus = prepare_dataset(test_sentences, mode='test', word_to_id=dic['word_to_id'], tag_to_id=dic['tag_to_id'])

--2025-03-19 18:26:59--  https://princeton-nlp.github.io/cos484/assignments/a2/eng.train
Resolving princeton-nlp.github.io (princeton-nlp.github.io)... 185.199.108.153, 185.199.111.153, 185.199.109.153, ...
Connecting to princeton-nlp.github.io (princeton-nlp.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3213441 (3.1M) [application/octet-stream]
Saving to: ‘eng.train’

eng.train           100%[===================>]   3.06M  --.-KB/s    in 0.05s   

2025-03-19 18:27:00 (62.1 MB/s) - ‘eng.train’ saved [3213441/3213441]

--2025-03-19 18:27:00--  https://princeton-nlp.github.io/cos484/assignments/a2/eng.val
Resolving princeton-nlp.github.io (princeton-nlp.github.io)... 185.199.108.153, 185.199.111.153, 185.199.109.153, ...
Connecting to princeton-nlp.github.io (princeton-nlp.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 774436 (756K) [application/octet-stream]
Saving to: ‘eng.val’


#### Experiment with an HMM with greedy decoding for Problem 2(a)

In [17]:
runHiddenMarkovModel(
    train_corpus = train_corpus,
    test_corpus = test_corpus,
    dic = dic,
    decode_type = 'greedy',
    output_file = None
)

Train results:
Tagging...


100%|██████████| 14041/14041 [00:02<00:00, 6513.61it/s]


---- 14041 lines tagged in 2.1585s ----
Overall accuracy: 0.9544742438157164

Tags:  {0: 'O', 1: 'PER', 2: 'ORG', 3: 'LOC', 4: 'MISC'}
Confusion Matrix:
 [[168060    135   1233     48    102]
 [  1999   8628    456     39      6]
 [  1632     98   7291    731    273]
 [   741     49    567   6886     54]
 [   665     32    206    204   3486]]
F-1 scores:  [0.98087109 0.85979073 0.73728385 0.84986115 0.81888654]

-----------
Test results:
Tagging...


100%|██████████| 3490/3490 [00:00<00:00, 8092.57it/s]


---- 3490 lines tagged in 0.4341s ----
Overall accuracy: 0.9241331540561464

Tags:  {0: 'O', 1: 'PER', 2: 'ORG', 3: 'LOC', 4: 'MISC'}
Confusion Matrix:
 [[40558    17   543    17    29]
 [ 1105  1294   267    16     8]
 [  606    30  1382   176    56]
 [  249    15   215  1478    18]
 [  233     8    79    37   650]]
F-1 scores:  [0.96664482 0.63838185 0.58361486 0.7991349  0.73529412]


#### Experiment with an HMM with viterbi decoding for Problem 2(b)

In [18]:
runHiddenMarkovModel(
    train_corpus = train_corpus,
    test_corpus = test_corpus,
    dic = dic,
    decode_type = 'viterbi',
    output_file = None
)

Train results:
Tagging...


  0%|          | 0/14041 [00:00<?, ?it/s]<ipython-input-11-c936f1159cc3>:137: RuntimeWarning: divide by zero encountered in log
  scores = M[i-1, :] + np.log(A[:, j]) + np.log(O[j, sentence[i]])
100%|██████████| 14041/14041 [00:07<00:00, 2002.59it/s]


---- 14041 lines tagged in 7.0140s ----
Overall accuracy: 0.9177540626949087

Tags:  {0: 'O', 1: 'PER', 2: 'ORG', 3: 'LOC', 4: 'MISC'}
Confusion Matrix:
 [[168440    724    275     26    113]
 [  4619   6412     62     29      6]
 [  4968    410   4169    332    146]
 [  2870     73    334   4972     48]
 [  1493     71     58     90   2881]]
F-1 scores:  [0.95713247 0.68147518 0.55873484 0.72341045 0.7399512 ]

-----------
Test results:
Tagging...


  0%|          | 0/3490 [00:00<?, ?it/s]<ipython-input-11-c936f1159cc3>:137: RuntimeWarning: divide by zero encountered in log
  scores = M[i-1, :] + np.log(A[:, j]) + np.log(O[j, sentence[i]])
100%|██████████| 3490/3490 [00:01<00:00, 2297.97it/s]

---- 3490 lines tagged in 1.5208s ----
Overall accuracy: 0.9096279998370207

Tags:  {0: 'O', 1: 'PER', 2: 'ORG', 3: 'LOC', 4: 'MISC'}
Confusion Matrix:
 [[40667   364    76    12    45]
 [ 1044  1593    33    13     7]
 [ 1280   101   726   104    39]
 [  708    29    77  1142    19]
 [  424    19    21    21   522]]
F-1 scores:  [0.95365061 0.66430359 0.45617342 0.69911234 0.63697376]


In [20]:
from prettytable import PrettyTable

def compare_decoding_methods(train_corpus, test_corpus, dic):
    # Initialize models for both decoding methods
    greedy_model = HMM(dic, 'greedy')
    viterbi_model = HMM(dic, 'viterbi')

    # Train both models
    print("Training models...")
    greedy_model.train(train_corpus)
    viterbi_model.train(train_corpus)

    # Select a sample sentence from test corpus
    sample_idx = 20  # You can change this to any index
    sample = test_corpus[sample_idx]

    # Get predictions from both models
    greedy_tags = greedy_model.tag(sample['words'])
    viterbi_tags = viterbi_model.tag(sample['words'])

    # Convert tag IDs to tag names
    greedy_str_tags = [dic['id_to_tag'][t] for t in greedy_tags]
    viterbi_str_tags = [dic['id_to_tag'][t] for t in viterbi_tags]
    true_str_tags = [dic['id_to_tag'][t] for t in sample['tags']]

    # Create a pretty table to display results
    table = PrettyTable()
    table.field_names = ["Word", "True Tag", "Greedy Prediction", "Viterbi Prediction"]

    # Add rows to the table
    for i in range(len(sample['str_words'])):
        word = sample['str_words'][i]
        true_tag = true_str_tags[i]
        greedy_tag = greedy_str_tags[i]
        viterbi_tag = viterbi_str_tags[i]

        # Highlight differences
        if greedy_tag != true_tag and viterbi_tag == true_tag:
            greedy_tag = f"*{greedy_tag}*"
        elif viterbi_tag != true_tag and greedy_tag == true_tag:
            viterbi_tag = f"*{viterbi_tag}*"
        elif viterbi_tag != true_tag and greedy_tag != true_tag:
            greedy_tag = f"*{greedy_tag}*"
            viterbi_tag = f"*{viterbi_tag}*"

        table.add_row([word, true_tag, greedy_tag, viterbi_tag])

    # Print the table
    print("\nComparing Greedy vs Viterbi Decoding on a Sample Sentence:")
    print(table)

    # Compute overall accuracy for both methods on test data
    print("\nComputing overall accuracy on test data...")

    # For greedy decoding
    greedy_correct = 0
    total_tokens = 0
    for sentence in test_corpus:
        tags = greedy_model.tag(sentence['words'])
        greedy_correct += np.sum(np.array(tags) == np.array(sentence['tags']))
        total_tokens += len(sentence['words'])

    # For viterbi decoding
    viterbi_correct = 0
    for sentence in test_corpus:
        tags = viterbi_model.tag(sentence['words'])
        viterbi_correct += np.sum(np.array(tags) == np.array(sentence['tags']))

    # Create accuracy comparison table
    acc_table = PrettyTable()
    acc_table.field_names = ["Decoding Method", "Correct Predictions", "Total Tokens", "Accuracy"]
    acc_table.add_row(["Greedy", greedy_correct, total_tokens, f"{greedy_correct/total_tokens:.4f}"])
    acc_table.add_row(["Viterbi", viterbi_correct, total_tokens, f"{viterbi_correct/total_tokens:.4f}"])

    print("\nOverall Accuracy Comparison:")
    print(acc_table)

# Add this cell and run it after training both models
compare_decoding_methods(train_corpus, test_corpus, dic)

Training models...


<ipython-input-11-c936f1159cc3>:137: RuntimeWarning: divide by zero encountered in log
  scores = M[i-1, :] + np.log(A[:, j]) + np.log(O[j, sentence[i]])



Comparing Greedy vs Viterbi Decoding on a Sample Sentence:
+----------------+----------+-------------------+--------------------+
|      Word      | True Tag | Greedy Prediction | Viterbi Prediction |
+----------------+----------+-------------------+--------------------+
|    Somerset    |   ORG    |        ORG        |        *O*         |
|       00       |    O     |         O         |         O          |
|      and       |    O     |         O         |         O          |
|      000       |    O     |         O         |         O          |
|       (        |    O     |         O         |         O          |
|       P.       |   PER    |        PER        |        *O*         |
|    Simmons     |   PER    |        PER        |        PER         |
|      0-00      |    O     |         O         |         O          |
|       )        |    O     |         O         |         O          |
|       ,        |    O     |         O         |         O          |
| Leicestershire 